In [1]:

from google.oauth2.service_account import Credentials
import openai
import httplib2
from googleapiclient.discovery import build
from google.oauth2 import service_account

In [2]:

# Dictionary to store the placeholder replacements
placeholder_replacements = {
    'name': "Sulemaan Farooq",
    'email': "shf46@rutgers.edu",
    'phone': "+1234567890",
    'city': "Cityville",
    'state': "Stateville",
    'linkedin_link': "https://www.linkedin.com/in/sulemaanfarooq/",
    # 'resume_link': "https://www.example.com/resume",
    # 'github_link': "https://github.com/sulemaan",
    'university': "University of Example",
    'university start date': "September 2018",
    'university end date': "May 2022",
    'major': "Computer Science",
    'minor': "Business Administration",
    'spoken_languages': "English, Spanish",
    'languages': "Python, Java, JavaScript",
    'technologies': "React, Django, MongoDB",
    'leadership': "President, Computer Science Club",
    'degree_type': "Bachelor of Science",
}

placeholder_replacements.update({
    'experience1': "Software Developer Intern",
    'title1': "ABC Tech Solutions",
    'experience1 start date': "June 2021",
    'experience1 end date': "August 2021",
    'experience1 location': "Cityville, State",
    'experience1 bullet1': "Developed and maintained web applications using Django and React.",
    'experience1 bullet2': "Collaborated with cross-functional teams to deliver high-quality software.",
    'experience1 bullet3': "Participated in daily stand-up meetings and sprint planning sessions.",
})

placeholder_replacements.update({
    'project1': "E-commerce Website",
    'skills1': "React, Node.js, MongoDB",
    'project1 bullet1': "Designed and implemented the front-end of an e-commerce website using React.",
    'project1 bullet2': "Developed RESTful APIs and database models using Node.js and MongoDB.",
})

# Path to your service account credentials JSON file
SERVICE_ACCOUNT_FILE = '/Users/cheesenaan/Documents/projects/resume_app/project/.ipynb_checkpoints/resume_App/resume_app/resume_app/doc.json'
#SERVICE_ACCOUNT_FILE = '/home/displayai/displayai/resume_app/doc.json'

# ID of the Google Doc you want to modify
DOCUMENT_ID = '1PVKqAkOTjdorBmlJOIGHYhIjYuBPxuPZzKeN4TUQZNE'

# Folder ID where the copy will be placed
FOLDER_ID = '1aGcF78a65Nus-K9kCv8P7NzMBSZjJnNY'

# Authenticate and create the Google Drive API service
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive'])
service = build('drive', 'v3', credentials=credentials)

# Create a copy of the document
copy_metadata = {
    'name':  'Sulemaan - Resume',
    'parents': [FOLDER_ID]
}
copy_response = service.files().copy(fileId=DOCUMENT_ID,  body=copy_metadata).execute()
copy_id = copy_response['id']

print(f"Copy created with ID: {copy_id}")

DOCUMENT_ID = str(copy_id)


# Authenticate and create the Google Docs API service
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/documents'])
service = build('docs', 'v1', credentials=credentials)


# Get the content of the Google Doc
doc = service.documents().get(documentId=DOCUMENT_ID).execute()

# Process each content element and replace placeholders
for content in doc['body']['content']:
    if 'paragraph' in content:
        elements = content['paragraph']['elements']
        for element in elements:
            if 'textRun' in element:
                text_run = element['textRun']
                if 'content' in text_run:
                    content_text = text_run['content']
                    for placeholder, replacement in placeholder_replacements.items():
                        if placeholder in content_text:
                            content_text = content_text.replace(placeholder, replacement)
                            text_run['content'] = content_text

# Create the requests list for batch updating
requests = []
for placeholder, replacement in placeholder_replacements.items():
    requests.append({
        'replaceAllText': {
            'containsText': {
                'text': '{{' + placeholder + '}}',
                'matchCase': False
            },
            'replaceText': replacement
        }
    })

# Execute the batch update requests
result = service.documents().batchUpdate(documentId=DOCUMENT_ID,  body={'requests': requests}).execute()

print('Placeholders replaced successfully in the Google Doc')
# Get the content of the Google Doc
doc = service.documents().get(documentId=DOCUMENT_ID).execute()
body_content = doc['body']['content']

# Iterate over the paragraphs in reverse order
requests = []
for i in range(len(body_content) - 1, -1, -1):
    paragraph = body_content[i]
    if 'paragraph' in paragraph:
        elements = paragraph['paragraph']['elements']
        contains_template_text = any('{{' in element['textRun']['content'] and '}}' in element['textRun']['content'] for element in elements)
        if contains_template_text:
            # Remove the paragraph if it contains {{}} template text
            requests.append({
                'deleteContentRange': {
                    'range': {
                        'startIndex': paragraph['startIndex'],
                        'endIndex': paragraph['endIndex']
                    }
                }
            })

# Execute the batch update requests to remove paragraphs with unfilled {{}} template text
if requests:
    service.documents().batchUpdate(documentId=DOCUMENT_ID, body={'requests': requests}).execute()
    print("Lines with unfilled {{}} template text removed successfully!")


def execute_remove_blank_lines():
    # Execute the removeBlankLines function in the Google Doc
    requests = [
        {
            'runFunction': {
                'function': 'removeEmptyLines',
            },
        }
    ]

    result = service.documents().batchUpdate(documentId=DOCUMENT_ID, body={'requests': requests}).execute()

    print('removeBlankLines function executed successfully!')


# Get the link of the Google Doc
resume_link = f"https://docs.google.com/document/d/{DOCUMENT_ID}"
print("Link to the Google Doc:", resume_link)


Copy created with ID: 15SavP_3CQrRMn99_II5Iz9idc5ob94Zge5KnI4uqRVg
Placeholders replaced successfully in the Google Doc
Lines with unfilled {{}} template text removed successfully!
Link to the Google Doc: https://docs.google.com/document/d/15SavP_3CQrRMn99_II5Iz9idc5ob94Zge5KnI4uqRVg


In [30]:
def remove_blank_lines(document_id):
    # Get the content of the Google Doc
    doc = service.documents().get(documentId=document_id).execute()
    body = doc['body']

    # List to store delete requests
    delete_rows = []

    # Iterate through all elements in reverse order
    total_elements = len(body['content'])
    i = total_elements - 1
    while i >= 0:
        element = body['content'][i]

        # Check if the element is a paragraph
        if 'paragraph' in element:
            paragraph = element['paragraph']
            text = paragraph['elements'][0]['textRun']['content'].strip()

            # Check if the current element is empty and the previous element exists and is also empty
            if i != total_elements - 1 and len(text) == 0 and 'paragraph' in body['content'][i - 1] and \
                    body['content'][i - 1]['paragraph']['elements'][0]['textRun']['content'].strip() == '':
                delete_rows.append(i)

        i -= 1

    result = service.documents().batchUpdate(
    documentId=DOCUMENT_ID, body={'requests': requests}).execute()


# Call the remove_blank_lines function with your document ID
remove_blank_lines('15SavP_3CQrRMn99_II5Iz9idc5ob94Zge5KnI4uqRVg')


In [33]:
def remove_blank_lines(document_id):
    # Get the content of the Google Doc
    doc = service.documents().get(documentId=document_id).execute()
    body = doc['body']

    # List to store delete requests
    delete_start_indices = []
    delete_end_indices = []
    delete_rows = []

    # Iterate through all elements in reverse order
    total_elements = len(body['content'])
    i = total_elements - 1
    while i >= 0:
        element = body['content'][i]

        # Check if the element is a paragraph
        if 'paragraph' in element:
            paragraph = element['paragraph']
            text = paragraph['elements'][0]['textRun']['content'].strip()

            # Check if the current element is empty and the previous element exists and is also empty
            if i != total_elements - 1 and len(text) == 0 and 'paragraph' in body['content'][i - 1] and \
                    body['content'][i - 1]['paragraph']['elements'][0]['textRun']['content'].strip() == '':
                # Store the character index where the row starts
                start_index = paragraph['elements'][0]['startIndex']
                end_index = paragraph['elements'][0]['endIndex']
                delete_start_indices.append(start_index)
                delete_end_indices.append(end_index)
                delete_rows.append(i)

        i -= 1

    print(delete_rows)
    print(delete_start_indices)
    print(delete_end_indices)

    requests = [
        {
            'deleteContentRange': {
                'range': {
                    'startIndex': delete_start_indices[0],
                    'endIndex': delete_end_indices[0],
                }

            }

        },
    ]
    result = service.documents().batchUpdate(
        documentId=DOCUMENT_ID, body={'requests': requests}).execute()





# Call the remove_blank_lines function with your document ID
remove_blank_lines('15SavP_3CQrRMn99_II5Iz9idc5ob94Zge5KnI4uqRVg')


[25, 24, 23, 22, 16, 15, 14]
[918, 917, 916, 915, 725, 724, 723]
[919, 918, 917, 916, 726, 725, 724]


In [34]:
def remove_blank_lines(document_id):
    # Get the content of the Google Doc
    doc = service.documents().get(documentId=document_id).execute()
    body = doc['body']

    # Lists to store delete requests
    delete_start_indices = []
    delete_end_indices = []
    delete_rows = []

    # Iterate through all elements in reverse order
    total_elements = len(body['content'])
    i = total_elements - 1
    while i >= 0:
        element = body['content'][i]

        # Check if the element is a paragraph
        if 'paragraph' in element:
            paragraph = element['paragraph']
            text = paragraph['elements'][0]['textRun']['content'].strip()

            # Check if the current element is empty and the previous element exists and is also empty
            if i != total_elements - 1 and len(text) == 0 and 'paragraph' in body['content'][i - 1] and \
                    body['content'][i - 1]['paragraph']['elements'][0]['textRun']['content'].strip() == '':
                # Store the character index where the row starts
                start_index = paragraph['elements'][0]['startIndex']
                end_index = paragraph['elements'][0]['endIndex']
                delete_start_indices.append(start_index)
                delete_end_indices.append(end_index)
                delete_rows.append(i)

        i -= 1

    # Create a list of delete requests based on the stored start and end indices
    requests = [
        {
            'deleteContentRange': {
                'range': {
                    'startIndex': start_index,
                    'endIndex': end_index,
                }
            }
        }
        for start_index, end_index in zip(delete_start_indices, delete_end_indices)
    ]

    # Execute the batch update with delete requests
    result = service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()

# Call the remove_blank_lines function with your document ID
remove_blank_lines('15SavP_3CQrRMn99_II5Iz9idc5ob94Zge5KnI4uqRVg')
